<a href="https://colab.research.google.com/github/ronitroy05/AI-Translator/blob/main/LLM_3M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  Optimized Training for Kine-3M (3M Parameter LLM) on TPU v2-8


!pip install torch transformers datasets accelerate peft torch_xla

import torch
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch.optim as optim
from transformers import GPT2Config, AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType
from torch.utils.data import DataLoader

# 1: TPU set up
device = xm.xla_device()
print("Using device:", device)

# 2: Define Model Configuration (3M Parameters)
config = GPT2Config(
    vocab_size=4000,
    n_positions=512,
    n_embd=128,
    n_layer=2,
    n_head=2,
    intermediate_size=512,
)

#  3: Initialize the model
model = AutoModelForCausalLM.from_config(config)
model.to(device)
print(f"Model Parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.2f}M")

#  4: Load & Tokenize Dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_datasets.set_format(type="torch", columns=["input_ids"])

batch_size = 4  # Reduced for stability
train_dataloader = DataLoader(tokenized_datasets, batch_size=batch_size, shuffle=True)

# Apply LoRA (Optimized Training)
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=4,
    lora_alpha=16,
    lora_dropout=0.05
)

model = get_peft_model(model, peft_config)
model.to(device)

# Training Loop (Optimized for TPU)
optimizer = optim.AdamW(model.parameters(), lr=1e-4)  # Lowered LR for better training

print("\n🚀 Training Kine-3M on TPU!\n")
model.train()

num_epochs = 5  # Increase training since it's not learning well
best_loss = float("inf")

for epoch in range(num_epochs):
    para_loader = pl.MpDeviceLoader(train_dataloader, device)

    total_loss = 0
    for batch in para_loader:
        optimizer.zero_grad()

        inputs = batch["input_ids"].to(device)
        labels = batch["input_ids"].to(device)

        outputs = model(inputs, labels=inputs)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        xm.optimizer_step(optimizer)
        xm.mark_step()

    avg_loss = total_loss / len(train_dataloader)
    print(f"✅ Epoch {epoch + 1}/{num_epochs} - Loss: {avg_loss:.4f}")

    if avg_loss < best_loss:
        best_loss = avg_loss
    else:
        print("⚠️ Loss increased, reducing learning rate!")
        for param_group in optimizer.param_groups:
            param_group["lr"] *= 0.8  # Reduce LR to stabilize training

print("\n✅ Training Complete!\n")

# 7: Save Kine-3M Model Safely
model_path = "Kine-3M.pth"
torch.save(model.state_dict(), model_path)
print("✅ Model saved successfully!")

# 8: Download the Model
from google.colab import files
files.download(model_path)

#  9: Test the Model
def generate_text(prompt):
    model.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(inputs["input_ids"], max_length=100, temperature=0.7)
    return tokenizer.decode(output[0], skip_special_tokens=True)

print(generate_text("Once upon a time"))



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.1/231.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    

/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


Using device: xla:0
Model Parameters: 1.49M


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(



🚀 Training Kine-3M on TPU!

✅ Epoch 1/5 - Loss: nan
⚠️ Loss increased, reducing learning rate!
✅ Epoch 2/5 - Loss: nan
⚠️ Loss increased, reducing learning rate!
✅ Epoch 3/5 - Loss: nan
⚠️ Loss increased, reducing learning rate!
✅ Epoch 4/5 - Loss: nan
⚠️ Loss increased, reducing learning rate!
✅ Epoch 5/5 - Loss: nan
⚠️ Loss increased, reducing learning rate!

✅ Training Complete!

✅ Model saved successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
